# Import all the necessary packages

In [3]:
from pyspark import SparkContext
from operator import add
import re
import time

# Create a spark context which will be used for mapper and reducer 

In [5]:
sc = SparkContext.getOrCreate()

In [1]:
# Process the lemmatizer file and make a hashMap of having keys as the lemma and its corresponsding values as the words

lemmaWords = {}
with open( "new_lemmatizer.csv", "r", encoding = 'utf-8') as f:
    words = []
    count = 0
    for line in f:
        array = line.split(",")
        lemma = array[0]
        i = 1
        while i < len(array):
            if(array[i] != "" and array[i] != "\n"):
                words.append(array[i])
            i+=1
        lemmaWords[lemma] = words
        words = []

# Function to the 2gram word cooccurrence pairs

def getTwoGramWordPairs(line):
    if len(line.split(">")) > 1:
        location = line.split(">")[0] + ">"
        words_array = re.sub(r'([^\s\w]|_)+', "",line.split(">")[1].strip()).split(" ")
        print(words_array)

        for i in range(0, len(words_array)-2):
            for j in range(i+1, len(words_array)-1):
                if words_array[i] != "" and words_array[j] != "":
                    if words_array[i] in lemmaWords and words_array[j] in lemmaWords:
                        for lemma1 in lemmaWords[words_array[i]]:
                            for lemma2 in lemmaWords[str(words_array[j])]:
                                yield (lemma1+"-"+lemma2, location)
                    elif words_array[i] in lemmaWords:
                        lemma2 = words_array[j]
                        for lemma1 in lemmaWords[words_array[i]]:
                            yield (lemma1+"-"+lemma2, location)
                    elif words_array[j] in lemmaWords:                

                        lemma1 = words_array[i]
                        for lemma2 in lemmaWords[words_array[j]]:
                            yield (lemma1+"-"+lemma2, location)
                    else:
                        yield (words_array[i]+"-"+words_array[j], location)



# FLatMap is used as the mapper and reducebyKey is used as the reducer

In [6]:
start_time = time.time()

words = sc.textFile("D:\DIC\Lab5\Files\*.tess")

words.flatMap(getTwoGramWordPairs).take(10)
counts = words.flatMap(getTwoGramWordPairs).reduceByKey(lambda x, y: x + " " + y)

print("Execution time --- %s seconds" % (time.time() - start_time))

Execution time --- 14.132959127426147 seconds


# Write the cooccurence pairs into a file

In [5]:
cooccurrence = counts.collect()
f = open('2grams_output', 'w')

for i in cooccurrence:
    f.write(str(i)+"\n")
    
f.close()    